Se empieza importando el dataset. Este dataset es de ventas de juegos, en sus columnas incluye el nombre del juego, la consola en la que fue lanzado, el año, el genero, la editora y las ventas en norteamerica, europa, japon, resto del mundo y finalmente ventas globales. En este caso la columna objetivo será la plataforma o consola para la que fue lanzado un juego, utilizando como datos las ventas, año, genero y editora.
Se utilizó aprendizaje supervisado.

Link dataset: https://www.kaggle.com/gregorut/videogamesales 

In [ ]:
import numpy as np 
import pandas as pd 

df = pd.read_csv ('../input/videogamesales/vgsales.csv')
pd.set_option('max_columns', None)
df = df.replace(np.nan,"0")

A continuación se inicia con el preprocesamiento. Para las columnas categoricas, como plataforma(consola), editora y genero del juego se utilizó LabelEncoder y para las columnas numéricas, que incluyen todas las columnas de ventas, se utilizó MinMaxScaler para llevar los datos a un rango, 0 a 100 en este caso y StandardScaler para estandarizar las columnas y tener media = 0 y varianza = 1.

In [ ]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
df['Publisher'] = encoder.fit_transform(df.Publisher.values)
df['Genre'] = encoder.fit_transform(df.Genre.values)

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))
df.iloc[:,6:11] = min_max_scaler.fit_transform(df.iloc[:,6:11].values)

scaler = preprocessing.StandardScaler()
df.iloc[:,6:11] = scaler.fit_transform(df.iloc[:,6:11].values)

Posteriormente seleccionamos las columnas de datos y la de objetivo. El objetivo será platform(consola o plataforma) y las de datos serán las columnas de ventas, año, editora y genero.

In [ ]:
X = df[['Genre','Publisher','Year','NA_Sales','JP_Sales','EU_Sales','Other_Sales','Global_Sales']]
y = df['Platform']

A continuación se prepara train y test, el 20% del dataset quedará para realizar las pruebas y el 80% restante se utilizará para entrenar el clasificador. También se prepara KFold que separará el split en 10 y realizará las prubeas en cada división. Comentado queda RepeatedKFold, que repite KFold n veces. 

In [ ]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
#kf = model_selection.RepeatedKFold(n_splits=10, n_repeats=3)   RepeatedKfold
kf = model_selection.KFold(n_splits=10)

A continuación se prepara y utliza el clasificador. Se utiliza RandomForestClassifier, despues de probar con varios clasificadores, los que daban mejores resultados fueron RandomForestClassifier y DecisionTreeClassifier. Jugando con los parametros, no variaba mucho, pero el mejor resultado fue con entropia como criterio de RandomForest. 
RandomForest crea varios DecisionTrees, cada árbol da una clasificación y el resultado es la clase con mayor número de votos en todo el bosque. Si se busca precisión RandomForest es una de las mejores opciones.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clasificador = RandomForestClassifier(criterion='entropy')
clasificador = clasificador.fit(X_train, y_train)
predictions = clasificador.predict(X_test)

A continuación se imprime el puntaje de acierto de cada división del dataset realizada con KFold.

In [ ]:
scores = model_selection.cross_val_score(clasificador, X_train, y_train, cv=kf, scoring="accuracy")
print("Precision K-fold:", scores)

Finalmente se imprime el reporte de clasificación de la clasificación realizada y la matriz de confusión. La clasificación siempre ronda el 60% de acierto, con el clasificador que mejor funciona. Esto puede ser porque las ventas de juegos muchas veces pueden ser impredecibles. Sobre todo en consolas más recientes como PS3, PS4, Xbox 360 o Xbox One los generos que venden mucho un año, o la editora que fue exitosa un año puede tener un bajón al año siguiente y juegos que venden muy bien en Japón, pueden no vender bien en Europa y Norte América o viceversa. 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

reporte = classification_report(y_test,predictions)
matriz = confusion_matrix(y_test,predictions)

print(reporte)
#print(matriz)

Para hacer el clasificador utilizando Pipeline, primero se importa el dataset y se realiza la división de columnas de datos y columna objetivo. 

In [ ]:
df = pd.read_csv ('../input/videogamesales/vgsales.csv')
pd.set_option('max_columns', None)
df = df.replace(np.nan,"0")

X = df[['Publisher', 'Genre','Year','NA_Sales','JP_Sales','EU_Sales','Other_Sales','Global_Sales']]
y = df['Platform']

para las columnas numericas se utiliza el mismo preprocesamiento, MinMaxScaler, StandardScaler y se realiza un Pipeline para preprocesar estas columnas. Para las categoricas solo se utiliza OneHotEncoder, porque LabelEncoder da problemas con el número de parametros.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
numericas = ['Year','NA_Sales','JP_Sales','EU_Sales','Other_Sales','Global_Sales']
numeric_transformer = Pipeline(steps=[('minmax',MinMaxScaler()),('scaler', StandardScaler())])

from sklearn.preprocessing import OneHotEncoder
categoricas = ['Publisher', 'Genre']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

A continuación se utiliza ColumnTransformer para unir las columnas categoricas y numéricas. 

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, categoricas),('num', numeric_transformer, numericas)])

Se prepara el pipeline con el columntransformer y el clasificador, se utilizó DecisionTree para ver la diferencia con RandomForest. A continuación se realiza la separación para train y test.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Finalmente se realiza la clasificación y se imprime el puntaje. Un acierto menor pero se acerca a la precisión de RandomForest.

In [ ]:
pipe.fit(X_train, y_train)
print('Puntaje clasificador:', pipe.score(X_test, y_test))